## Для получения спарк сессии достаточно исполнить


In [181]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [182]:
given_cources = [[23126, u'en', u'Compass - powerful SASS library that makes your life easier'], 
[21617, u'en', u'Preparing for the AP* Computer Science A Exam \u2014 Part 2'], 
[16627, u'es', u'Aprende Excel: Nivel Intermedio by Alfonso Rinsche'], 
[11556, u'es', u'Aprendizaje Colaborativo by UNID Universidad Interamericana para el Desarrollo'], 
[16704, u'ru', u'\u041f\u0440\u043e\u0433\u0440\u0430\u043c\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u0438\u0435 \u043d\u0430 Lazarus'], 
[13702, u'ru', u'\u041c\u0430\u0442\u0435\u043c\u0430\u0442\u0438\u0447\u0435\u0441\u043a\u0430\u044f \u044d\u043a\u043e\u043d\u043e\u043c\u0438\u043a\u0430']]

In [183]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

schema = StructType(fields=[
    StructField("id", IntegerType()),
    StructField("lang", StringType()),
    StructField("desc", StringType())
])

In [184]:
df = spark.createDataFrame(data=given_cources, schema=schema)

In [185]:
df.show() #посмотрю примеры

+-----+----+--------------------+
|   id|lang|                desc|
+-----+----+--------------------+
|23126|  en|Compass - powerfu...|
|21617|  en|Preparing for the...|
|16627|  es|Aprende Excel: Ni...|
|11556|  es|Aprendizaje Colab...|
|16704|  ru|Программирование ...|
|13702|  ru|Математическая эк...|
+-----+----+--------------------+



In [186]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, Normalizer, StopWordsRemover
from pyspark.ml import Pipeline
import pyspark.sql.functions as f
from pyspark.sql.functions import pandas_udf, udf, col, isnan, isnull, broadcast, desc, lower
from pyspark.sql.types import FloatType, ArrayType, StringType, IntegerType
import json
import re

In [187]:
@f.pandas_udf(ArrayType(StringType()))
# препроцессинг + токенизация 
def clear_string(series):
    regex = re.compile(u'[\w\d]{2,}', re.U)
    res = series.str.findall(regex)
    res_low = res.apply(lambda x: [v.lower() for v in x], ArrayType(StringType()))
    return res_low

In [188]:
df = df.withColumn('tokens', clear_string('desc'))

In [189]:
all_data = spark.read.json("/labs/slaba02/DO_record_per_line.json")

In [190]:
all_data = all_data.withColumn('tokens', clear_string('desc'))

In [191]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

hashingTF = HashingTF(inputCol="tokens", outputCol="rawFeatures", numFeatures=10000)
featurizedData = hashingTF.transform(all_data)
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

In [192]:
def cos_sim(a,b):
    return float(a.dot(b) / (a.norm(2) * b.norm(2)))

cos_sim_udf = udf(cos_sim, FloatType())

Рассчитываю косинусную близость и сохраняю в json по шагам: 
1. вывожу sparse-vector для каждого примера,
2. добавляю столбцом в датафрейм,
3. добавляю столбцом cos_sim для каждого примера,
4. вывожу столбец с топ-10 рекомендаций,
5. сохраняю json

In [193]:
# вывожу sparse-vector для каждого примера,
c21617 = rescaledData.select(col("features").alias("c21617")).where("id = 21617").limit(1)
c23126 = rescaledData.select(col("features").alias("c23126")).where("id = 23126").limit(1)
c16627 = rescaledData.select(col("features").alias("c16627")).where("id = 16627").limit(1)
c11556 = rescaledData.select(col("features").alias("c11556")).where("id = 11556").limit(1)
c16704 = rescaledData.select(col("features").alias("c16704")).where("id = 16704").limit(1)
c13702 = rescaledData.select(col("features").alias("c13702")).where("id = 13702").limit(1)

In [194]:
# добавляю столбцом в датафрейм,
rescaledData = rescaledData.crossJoin(c21617)
rescaledData = rescaledData.crossJoin(c23126)
rescaledData = rescaledData.crossJoin(c16627)
rescaledData = rescaledData.crossJoin(c11556)
rescaledData = rescaledData.crossJoin(c16704)
rescaledData = rescaledData.crossJoin(c13702)


In [195]:
# добавляю столбцом cos_sim для каждого примера,
rescaledData = rescaledData.withColumn("cos_sim_c21617", cos_sim_udf(col("features"), col("c21617")))
rescaledData = rescaledData.withColumn("cos_sim_c23126", cos_sim_udf(col("features"), col("c23126")))
rescaledData = rescaledData.withColumn("cos_sim_c16627", cos_sim_udf(col("features"), col("c16627")))
rescaledData = rescaledData.withColumn("cos_sim_c11556", cos_sim_udf(col("features"), col("c11556")))
rescaledData = rescaledData.withColumn("cos_sim_c16704", cos_sim_udf(col("features"), col("c16704")))
rescaledData = rescaledData.withColumn("cos_sim_c13702", cos_sim_udf(col("features"), col("c13702")))

In [196]:
# вывожу столбец с топ-10 рекомендаций

ans_21617 = (
rescaledData.select(col("id"))
    .where("lang = 'en'")
    .where("id != 21617")
    .where("cos_sim_c21617 is not null")
    .where("cos_sim_c21617 != 'NaN'")
    .orderBy("cos_sim_c21617", ascending=False)
    .limit(10).toPandas()
)
lst_21617 = [x for x in ans_21617.id]
lst_21617

[21609, 21616, 21608, 22298, 21628, 21630, 21623, 21508, 21081, 19417]

In [197]:
ans_23126 = (
rescaledData.select(col("id"))
    .where("lang = 'en'")
    .where("id != 23126")
    .where("cos_sim_c23126 is not null")
    .where("cos_sim_c23126 != 'NaN'")
    .orderBy("cos_sim_c23126", ascending=False)
    .limit(10).toPandas()
)
lst_23126 = [x for x in ans_23126.id]
lst_23126

[14760, 13665, 13782, 20638, 24419, 15909, 2724, 25782, 17499, 13348]

In [198]:
ans_16627 = (
rescaledData.select(col("id"))
    .where("lang = 'es'")
    .where("id != 16627")
    .where("cos_sim_c16627 is not null")
    .where("cos_sim_c16627 != 'NaN'")
    .orderBy("cos_sim_c16627", ascending=False)
    .limit(10).toPandas()
)
lst_16627 = [x for x in ans_16627.id]
lst_16627

[11431, 11575, 12247, 17964, 5687, 17961, 16694, 12660, 25010, 5558]

In [199]:
ans_11556 = (
rescaledData.select(col("id"))
    .where("lang = 'es'")
    .where("id != 11556")
    .where("cos_sim_c11556 is not null")
    .where("cos_sim_c11556 != 'NaN'")
    .orderBy("cos_sim_c11556", ascending=False)
    .limit(10).toPandas()
)
lst_11556 = [x for x in ans_11556.id]
lst_11556

[16488, 468, 13461, 23357, 19330, 7833, 9289, 10447, 22710, 11340]

In [200]:
ans_16704 = (
rescaledData.select(col("id"))
    .where("lang = 'ru'")
    .where("id != 16704")
    .where("cos_sim_c16704 is not null")
    .where("cos_sim_c16704 != 'NaN'")
    .orderBy("cos_sim_c16704", ascending=False)
    .limit(10).toPandas()
)
lst_16704 = [x for x in ans_16704.id]
lst_16704

[1236, 1247, 1365, 20288, 1273, 1164, 8186, 1233, 8203, 875]

In [201]:
ans_13702 = (
rescaledData.select(col("id"))
    .where("lang = 'ru'")
    .where("id != 13702")
    .where("cos_sim_c13702 is not null")
    .where("cos_sim_c13702 != 'NaN'")
    .orderBy("cos_sim_c13702", ascending=False)
    .limit(10).toPandas()
)
lst_13702 = [x for x in ans_13702.id]
lst_13702

[864, 21079, 8313, 1041, 28074, 8300, 1033, 13057, 21025, 1111]

In [202]:
# сохраняю json
data = {"21617": lst_21617, "23126": lst_23126, "11556": lst_11556, "16627": lst_16627, "13702": lst_13702, "16704": lst_16704} #собираю словарь для оформления ответа

import json
with open('lab02.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [203]:
sc.stop()